# H5: Калибровка и пороги

**H5 (Tech):** Калибровка вероятностей + пороги по требуемой точности (например, Precision≥0.9) снижает операционные издержки.

In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features, customer_velocity
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df = load_transactions(TX); fx = load_fx(FX)
df = convert_to_usd(df, fx); df = unpack_last_hour_activity(df); df = add_basic_time_features(df); df = customer_velocity(df)

train, test = split_time_aware(df)
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])

base = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
p_base = base.predict_proba(X_te)[:,1]
eval_base = eval_pack(y_te, p_base)

cal = CalibratedClassifierCV(base, method='isotonic', cv=3)
cal.fit(X_tr, y_tr)
p_cal = cal.predict_proba(X_te)[:,1]
eval_cal = eval_pack(y_te, p_cal)
eval_base, eval_cal

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

({'roc_auc': 0.5001122214983779,
  'pr_auc': 0.1980998640895631,
  'threshold_at_precision': 0.9,
  'thr_value': None,
  'recall_at_precision': None,
  'precision_achieved': None},
 {'roc_auc': 0.4991919410385204,
  'pr_auc': 0.19779356719251343,
  'threshold_at_precision': 0.9,
  'thr_value': None,
  'recall_at_precision': None,
  'precision_achieved': None})

### Выводы по H5 (калибровка и пороги)

**Факты**
- До калибровки: ROC-AUC ≈ **0.5001**, PR-AUC ≈ **0.19810**; порог при Precision ≥ 0.90 **не найден**.
- После изотонической калибровки: ROC-AUC ≈ **0.4992**, PR-AUC ≈ **0.19779**; порог при Precision ≥ 0.90 **не найден**.
- Разница в качестве пренебрежимо мала и находится на уровне случайных флуктуаций; ранжирование почти не изменилось.

**Интерпретация**
- Калибровка **не создаёт сигнал** — она лишь «выпрямляет» шкалу вероятностей, если у базовой модели уже есть информативное ранжирование. При отсутствии сигнала (ROC-AUC ~ 0.5, PR-AUC ~ prevalence) калибровка ожидаемо **не помогает** и может слегка ухудшать ранжирование из-за сглаживания.
- Недостижимость Precision ≥ 0.90 указывает на **слишком слабое ранжирование** вероятностей; сначала нужен прирост качества модели, затем — калибровка.

**Рекомендации**
1. **Сначала повысить качество модели**, затем калибровать:
   - усилить фичи (velocity без утечки, rarity/newness, взаимодействия CNP×CB×HRV, графовые признаки с историческими агрегатами);
   - смягчить OHE `min_frequency` (например, до 10) и проверить FX-конвертацию.
2. **Правильная процедура калибровки**:
   - использовать `CalibratedClassifierCV` c `cv='prefit'` и **отдельным валидационным срезом** (без переобучения на train);
   - сравнивать **ECE**/**Brier score** и строить **reliability curve** до/после калибровки.
3. **Пороговая политика**:
   - при недостижимом Precision ≥ 0.90 рассмотреть **сегментные пороги** (ночные часы, CNP, cross-border, high-risk) и/или временно снизить целевую точность на пилоте (например, до 0.85) с контролем доли фрикции.
   - параллельно — **квантильные пороги по amount_usd** как fallback до появления устойчивого модельного сигнала.

**Итог:** гипотеза H5 **не подтверждена на текущем качестве модели**. К калибровке вернёмся после улучшения ранжирования (рост PR-AUC над уровнем базовой доли fraud).